# Building blocks in Haystack: Data classes 

With Haystack, we can leverage the following built-in data classes: 

* Haystack Documents data class 

* Haystack ByteStream data class 

* Haystack ChatMessage data class 

* Haystack StreaminhChunk data class 

Each of these classes act as data structures that can be used to store and process data. We can use these classes to store data in a standardized format, and then use the Haystack API to process the data through data pipelines.

In the next section, we will provide examples of each.

### Haystack Documents data class 

The Document is a foundational data class in Haystack that encapsulates a variety of data types that can be queried, such as text snippets, tables, and binary data.

Let's import it and take a look at its functionality.

In [1]:
from haystack.preview.dataclasses import Document

help(Document)

Help on class Document in module haystack.preview.dataclasses.document:

class Document(builtins.object)
 |  Document(*args, **kwargs)
 |  
 |  Base data class containing some data to be queried.
 |  Can contain text snippets, tables, and file paths to images or audios.
 |  Documents can be sorted by score and saved to/from dictionary and JSON.
 |  
 |  :param id: Unique identifier for the document. When not set, it's generated based on the Document fields' values.
 |  :param content: Text of the document, if the document contains text.
 |  :param dataframe: Pandas dataframe with the document's content, if the document contains tabular data.
 |  :param blob: Binary data associated with the document, if the document has any binary data associated with it.
 |  :param meta: Additional custom metadata for the document. Must be JSON-serializable.
 |  :param score: Score of the document. Used for ranking, usually assigned by retrievers.
 |  :param embedding: Vector representation of the docu

Using the `help` function lets us see what parameters it accepts. 

### Let's create a simple Document object.

In [2]:
sample_document = Document(content="This is a simple document", meta={"name": "test_doc"})
sample_document

Document(id='ca53157e450d009adb4c2217111faadc9e7c02aefb22717c4901e1c1c1ba314a', content='This is a simple document', dataframe=None, blob=None, meta={'name': 'test_doc'}, score=None)

In [3]:
sample_document.id

'ca53157e450d009adb4c2217111faadc9e7c02aefb22717c4901e1c1c1ba314a'

We see that an id was automatically generated for the document. Let's create a dataframe-based document.

In [4]:
sample_document.content

'This is a simple document'

In [5]:
sample_document.meta

{'name': 'test_doc'}

### Let's create a dataframe-based Document

In [6]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups, load_iris

# Load some example data
iris_df = load_iris(as_frame=True)["frame"]
news_df = pd.DataFrame(fetch_20newsgroups(subset="train").data, columns=["text"])

# Save each row as a Document Object
iris_docs = [Document(dataframe=row.to_frame().T) for _, row in iris_df.iterrows()]

We see that each row was converted into a Document object, each with its own id. Let's access the first Document  and attributes.

In [7]:
iris_docs[0]

Document(id='22cf9396b67c1929c273ed65a6fcea5b8ba8b384ae45d5164be9ca7b6827c66c', content=None, dataframe=   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   

   target  
0     0.0  , blob=None, meta={}, score=None)

In [8]:
iris_docs[0].id

'22cf9396b67c1929c273ed65a6fcea5b8ba8b384ae45d5164be9ca7b6827c66c'

In [9]:
iris_docs[0].dataframe

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0


### Let's create a ByteStream-based data structure

The ByteStream class in Haystack represents a binary object that can be used within the API.

In [10]:
from haystack.preview.dataclasses import  ByteStream


In [11]:
# Assuming 'binary_data' is your binary data, for example, read from a file:
binary_data = b'Your binary data here'  # This could be the actual binary content, such as PDF or image data

# Convert binary data to ByteStream object
binary_blob = ByteStream(data=binary_data, mime_type='application/pdf')  # MIME type should match your data
binary_document = Document(blob=binary_blob, meta={"file_name": "example.pdf", "file_type": "PDF"})

In [12]:
binary_document.blob

ByteStream(data=b'Your binary data here', metadata={}, mime_type='application/pdf')

In [13]:
binary_document.id

'93c323201fc3b8509e51056dff8baee6ca9dec1c22cf2ce2f6cfc0bb04397c14'

### Let's create a ChatMessage Document

ChatMessage comes built in with the following roles

```python
class ChatRole(str, Enum):
    """Enumeration representing the roles within a chat."""

    ASSISTANT = "assistant"
    USER = "user"
    SYSTEM = "system"
    FUNCTION = "function"
```

These can be mapped to the roles present in OpenAI's GPT models. 

Read more https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide

In [14]:
from haystack.preview.dataclasses import ChatMessage

help(ChatMessage)

Help on class ChatMessage in module haystack.preview.dataclasses.chat_message:

class ChatMessage(builtins.object)
 |  ChatMessage(content: str, role: haystack.preview.dataclasses.chat_message.ChatRole, name: Optional[str], metadata: Dict[str, Any] = <factory>) -> None
 |  
 |  Represents a message in a LLM chat conversation.
 |  
 |  :param content: The text content of the message.
 |  :param role: The role of the entity sending the message.
 |  :param name: The name of the function being called (only applicable for role FUNCTION).
 |  :param metadata: Additional metadata associated with the message.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, content: str, role: haystack.preview.dataclasses.chat_message.ChatRole, name: Optional[str], metadata: Dict[str, Any] = <factory>) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  is_from(s

In [15]:
# Create a message from the assistant
assistant_msg = ChatMessage.from_assistant(content="Hello, how can I assist you today?")

# Create a message from the user
user_msg = ChatMessage.from_user(content="Can you show me the weather forecast?")

# Create a system message, for instance, to indicate that a user has joined the chat
system_msg = ChatMessage.from_system(content="A new user has joined the chat.")

# Create a function message, for example, to execute a command to retrieve weather data
function_msg = ChatMessage.from_function(content="Retrieving weather data...", name="fetch_weather")

print(assistant_msg)
print(user_msg)
print(system_msg)
print(function_msg)


ChatMessage(content='Hello, how can I assist you today?', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, metadata={})
ChatMessage(content='Can you show me the weather forecast?', role=<ChatRole.USER: 'user'>, name=None, metadata={})
ChatMessage(content='A new user has joined the chat.', role=<ChatRole.SYSTEM: 'system'>, name=None, metadata={})
ChatMessage(content='Retrieving weather data...', role=<ChatRole.FUNCTION: 'function'>, name='fetch_weather', metadata={})


In [16]:
user_message_doct = Document(content = user_msg)

user_message_doct.id

'740dcdab24b6c171e89af1f1158056d6f09c6cd238a39866dfe7160a47eeba9a'

In [17]:
user_message_doct.content

ChatMessage(content='Can you show me the weather forecast?', role=<ChatRole.USER: 'user'>, name=None, metadata={})

### StreamingChunk  data class

The StreamingChunk class is designed to manage segments of streamed content, which could be part of a larger message or data transfer in a streaming context.

In [18]:
from haystack.preview.dataclasses import StreamingChunk

In [19]:
help(StreamingChunk)

Help on class StreamingChunk in module haystack.preview.dataclasses.streaming_chunk:

class StreamingChunk(builtins.object)
 |  StreamingChunk(content: str, metadata: Dict[str, Any] = <factory>) -> None
 |  
 |  The StreamingChunk class encapsulates a segment of streamed content along with
 |  associated metadata. This structure facilitates the handling and processing of
 |  streamed data in a systematic manner.
 |  
 |  :param content: The content of the message chunk as a string.
 |  :param metadata: A dictionary containing metadata related to the message chunk.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, content: str, metadata: Dict[str, Any] = <factory>) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |

Here's an example of how to create an instance of the StreamingChunk data class, which might represent a segment of a live video stream or an ongoing audio broadcast:

In [23]:
# Metadata for the streaming chunk
stream_metadata = {
    "timestamp": "2023-11-08T12:00:00Z",
    "stream_id": "stream123",
    "segment_number": 1
}

# Content of the streaming chunk
stream_content = "This is the first segment of the live stream."

# Create the StreamingChunk instance
streaming_chunk = StreamingChunk(content=stream_content, metadata=stream_metadata)

print(streaming_chunk)


StreamingChunk(content='This is the first segment of the live stream.', metadata={'timestamp': '2023-11-08T12:00:00Z', 'stream_id': 'stream123', 'segment_number': 1})


We can generate metadata for the StreamingChunk object and capture changes in the stream. We can also store streaming content into Documents.

In [29]:
streaming_document = Document(content = stream_content, meta = stream_metadata)

streaming_document.id

'55b3a7072cc30c752c726922b929f073bf377fb72dbe89431c323031cf5360cd'

In [32]:
streaming_document.content

'This is the first segment of the live stream.'

### The `DocumentStore` class 

The `DocumentStore` class is an internal component of the Haystack library that serves as a registry for classes that are marked as document stores. A document store in Haystack is a place where documents are stored and retrieved, typically used as part of a pipeline to handle data for search and retrieval tasks. 

Let's begin saving our documents into a DocumentStore.


In [39]:
from haystack.preview.document_stores.in_memory.document_store import InMemoryDocumentStore

Recall our iris dataframe collection of Documents

In [33]:
iris_docs[0:5]

[Document(id='22cf9396b67c1929c273ed65a6fcea5b8ba8b384ae45d5164be9ca7b6827c66c', content=None, dataframe=   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
 0                5.1               3.5                1.4               0.2   
 
    target  
 0     0.0  , blob=None, meta={}, score=None),
 Document(id='c4852f58c6c65daaa7b11d7c009d8cbf7198c52c55f63fe27bf888beec64b673', content=None, dataframe=   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
 1                4.9               3.0                1.4               0.2   
 
    target  
 1     0.0  , blob=None, meta={}, score=None),
 Document(id='109c0409cdbcf2343ee97efd3ec334e74e73b5eeed3ecc362cbcff8adda10603', content=None, dataframe=   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
 2                4.7               3.2                1.3               0.2   
 
    target  
 2     0.0  , blob=None, meta={}, score=None),
 Document(id='3eef63e5

We will initialize an InMemoryDocumentStore and save our documents into it. From its documentation:

* Stores data in-memory. It's ephemeral and cannot be saved to disk.
* Uses the BM25 algorithm for document search by default.
* Useful for testing and quick prototyping.


In [44]:
help(InMemoryDocumentStore)

Help on class InMemoryDocumentStore in module haystack.preview.document_stores.in_memory.document_store:

class InMemoryDocumentStore(builtins.object)
 |  InMemoryDocumentStore(bm25_tokenization_regex: str = '(?u)\\b\\w\\w+\\b', bm25_algorithm: Literal['BM25Okapi', 'BM25L', 'BM25Plus'] = 'BM25Okapi', bm25_parameters: Optional[Dict] = None, embedding_similarity_function: Literal['dot_product', 'cosine'] = 'dot_product')
 |  
 |  Stores data in-memory. It's ephemeral and cannot be saved to disk.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, bm25_tokenization_regex: str = '(?u)\\b\\w\\w+\\b', bm25_algorithm: Literal['BM25Okapi', 'BM25L', 'BM25Plus'] = 'BM25Okapi', bm25_parameters: Optional[Dict] = None, embedding_similarity_function: Literal['dot_product', 'cosine'] = 'dot_product')
 |      Initializes the DocumentStore.
 |      
 |      :param bm25_tokenization_regex: The regular expression used to tokenize the text for BM25 retrieval.
 |      :param bm25_algorithm: The BM25 alg

In [34]:
# Write documents to document store
iris_docstore = InMemoryDocumentStore()
iris_docstore.write_documents(documents=iris_docs)


Counting total number of documents in the DocumentStore

In [36]:
iris_docstore.count_documents()

150

Transform DocumentStore into dictionary.

In [42]:
iris_docstore.to_dict()

{'type': 'InMemoryDocumentStore',
 'init_parameters': {'bm25_tokenization_regex': '(?u)\\b\\w\\w+\\b',
  'bm25_algorithm': 'BM25Okapi',
  'bm25_parameters': {},
  'embedding_similarity_function': 'dot_product'}}

In the next section, we will begin to get familiar with components and pipelines. This will enable us to process the data further, and connect the document store to a retriever and an LLM for data extraction using Natural Language.

[Follow next notebook](component.ipynb)